In [53]:
!pip install gurobipy
!pip install torch_geometric
#!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric


import numpy as np
import gurobipy as gp
from gurobipy import GRB
import torch
from sklearn.model_selection import train_test_split

import itertools
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.metrics import pairwise_distances as sklearn_pairwise_distances
import networkx as nx
 # Use "cosine" for cosine similarity
import heapq


os.environ["GRB_LICENSE_FILE"] = "gurobi (3).lic"
os.environ["GRB_WLSACCESSID"] = "f218200d-1f8d-4342-83f5-b7b2d9263751"  # Replace with your actual WLSACCESSID
os.environ["GRB_WLSSECRET"] = "528d596b-babc-4a1e-bda2-693c44f4f006"  # Replace with your actual WLSSECRET
os.environ["GRB_LICENSEID"] = "840285"  # Replace with your actual LICENSEID

from collections import Counter
from sklearn.metrics.pairwise import cosine_distances
from torch_geometric.datasets import WebKB, WikipediaNetwork, Actor

from torch_geometric.datasets import FacebookPagePage

from torch_geometric.datasets import SNAPDataset

import numpy as np
import torch
import random
import networkx as nx
from torch_geometric.datasets import WebKB
from collections import Counter, defaultdict
from sklearn.metrics import accuracy_score
from sortedcontainers import SortedSet  # Ordered set
from torch_geometric.utils import to_networkx

from torch_geometric.datasets import WikipediaNetwork

In [56]:


# 📌 Load the Cornell Dataset
dataset = WebKB(root='data', name='Cornell')
#dataset = WebKB(root='data', name='texas')
#dataset = WikipediaNetwork(root='data', name='chameleon')
#dataset = WikipediaNetwork(root='data', name='squirrel')
data = dataset[0]
data.edge_index = torch.cat([data.edge_index, data.edge_index.flip(0)], dim=1)

# ✅ Remove duplicate edges
data.edge_index = torch.unique(data.edge_index, dim=1)

# ✅ Convert to NetworkX Graph to Find Components
G = to_networkx(data, to_undirected=True)
components = list(nx.connected_components(G))
num_components = len(components)

print(f"\n✅ Loaded  dataset with {data.num_nodes} nodes, {data.edge_index.shape[1]} edges")
print(f"   - Features: {data.x.shape[1]}")
print(f"   - Number of Classes: {len(set(data.y.numpy()))}")
print(f"   - Number of Components: {num_components}")

# ✅ Set Training Percentage
train_percentage = 0.65

# ✅ Select Training Nodes
train_indices = []
for component in components:
    component = list(component)
    t = max(1, int(train_percentage * len(component)))
    sampled_nodes = np.random.choice(component, t, replace=False)
    train_indices.extend(sampled_nodes)

np.random.shuffle(train_indices)

# ✅ Initialize Prediction Labels
y_pred = np.full(data.num_nodes, -1)
for idx in train_indices:
    y_pred[idx] = data.y[idx].item()

# ✅ Compute Label Distribution
total_label_counts = Counter(data.y.numpy())
num_labels = len(set(data.y.numpy()))
alpha = 1  # Laplace smoothing

dataset_label_distribution = {
    lbl: (total_label_counts[lbl] + alpha) / (len(data.y) + alpha * num_labels)
    for lbl in total_label_counts.keys()
}

train_label_counts = Counter(y_pred[train_indices])

train_label_distribution = {
    lbl: (train_label_counts.get(lbl, 0) + alpha) / (len(train_indices) + alpha * num_labels)
    for lbl in dataset_label_distribution.keys()
}

# ✅ Precompute Neighbors for Each Node
A = torch.zeros((data.num_nodes, data.num_nodes))
A[data.edge_index[0], data.edge_index[1]] = 1
A = A.numpy()
neighbors_dict = {i: set(np.where(A[i] == 1)[0]) for i in range(data.num_nodes)}

# ✅ Track Instances Assigned to Each Label
class_instances = defaultdict(set)
for idx in train_indices:
    class_instances[data.y[idx].item()].add(idx)

# ✅ Ordered Set for Managing Unlabeled Vertices
ordered_set = SortedSet()
train_labeled_nodes = set(train_indices)

for node in range(data.num_nodes):
    if y_pred[node] == -1:  # Only process unlabeled nodes
        labeled_neighbors = [n for n in neighbors_dict[node] if y_pred[n] != -1]
        num_labeled_by_train = sum(1 for n in labeled_neighbors if n in train_labeled_nodes)
        num_labeled_by_propagation = len(labeled_neighbors) - num_labeled_by_train
        total_neighbors = len(neighbors_dict[node])

        if total_neighbors > 0:
            weighted_score = (num_labeled_by_propagation + 3 * num_labeled_by_train) / total_neighbors
        else:
            weighted_score = 0

        ordered_set.add((-weighted_score, node))

# ✅ Iteratively Label the Most Constrained Nodes
while ordered_set:
    _, node = ordered_set.pop(0)

    if y_pred[node] != -1:
        continue

    # Get labeled neighbors
    neighbor_labels = [y_pred[n] for n in neighbors_dict[node] if y_pred[n] != -1]
    label_counts = Counter(neighbor_labels)

    # 🚨 If this node has 0 labeled neighbors
    if not neighbor_labels:
        print(f"\n🚨 STRANGE: Node {node} was chosen, but it has 0 labeled neighbors!")
        break

    # Compute Neighbor Label Distribution
    neighbor_label_counts = Counter(neighbor_labels)
    total_labeled_neighbors = len(neighbor_labels)

    if total_labeled_neighbors > 0:
        neighbor_label_distribution = {
            lbl: neighbor_label_counts[lbl] / total_labeled_neighbors for lbl in neighbor_label_counts.keys()
        }
    else:
        neighbor_label_distribution = {lbl: 0 for lbl in dataset_label_distribution.keys()}

    # ✅ Compute Feature Distance to Each Class
    # feature_diffs = {}
    # for lbl in dataset_label_distribution.keys():
    #   if class_instances[lbl]:  # Only compute if we have instances of the label
    #       instance_features = data.x[list(class_instances[lbl])]
    #       node_feature = data.x[node].unsqueeze(0)

    #       # Compute distances for each instance
    #       distances = torch.norm(instance_features - node_feature, dim=1).tolist()

    #       # Apply weight based on whether the instance is from training or predicted labels
    #       weighted_distances = [
    #           (3 * dist) if instance in train_indices else dist  # Give 3x weight if in training set
    #           for instance, dist in zip(class_instances[lbl], distances)
    #       ]

    #       # Compute weighted average distance
    #       avg_distance = sum(weighted_distances) / len(weighted_distances)
    #       feature_diffs[lbl] = avg_distance
    #   else:
    #       feature_diffs[lbl] = 0  # No penalty if no instances exist

    feature_diffs = {}
    for lbl in dataset_label_distribution.keys():
        if class_instances[lbl]:  # Only compute if we have instances of the label
            instance_features = data.x[list(class_instances[lbl])]
            node_feature = data.x[node].unsqueeze(0)
            print(len(instance_features),len(node_feature))
            avg_distance = torch.mean(torch.norm(instance_features - node_feature, dim=1)).item()
            feature_diffs[lbl] = avg_distance
        else:
            feature_diffs[lbl] = 0  # No penalty if no instances exist

    # ✅ Select the Best Label Based on the Adjusted Score
    best_label_candidates = []
    max_value = float('-inf')
    a1=0.5
    a2=-6
    a3=-2
    a4=1
    a5=3
    for lbl in dataset_label_distribution.keys():
        score = (
            a1*train_label_distribution.get(lbl, 0)
            +a2* neighbor_label_distribution.get(lbl, 0)
            +a3*feature_diffs[lbl]  # Penalize by average feature distance
        )
        print(lbl,score,)

        if score - max_value > 0.01:
            best_label_candidates = [lbl]
            max_value = score
        elif abs(score - max_value) <= 0.01:
            best_label_candidates.append(lbl)


    best_label = random.choice(best_label_candidates)
    y_pred[node] = best_label
    class_instances[best_label].add(node)  # Track newly labeled nodes
    neighbor_labels = [y_pred[n] for n in neighbors_dict[node] if y_pred[n] != -1]
    label_counts = Counter(neighbor_labels)

    print(f"\n🔹 Labelling Node {node} ")
    # ✅ Print Prediction Outcome
    true_label = data.y[node].item()
    correct = "✅" if best_label == true_label else "❌"
    print(f"   -> Predicted Label: {best_label} | True Label: {true_label} {correct}")

    print(f"   - Total Neighbors: {len(neighbors_dict[node])}")
    print(f"   - Labeled Neighbors: {len(neighbor_labels)}")
    for lbl, count in label_counts.items():
        print(f"   - Class {lbl}: {count} neighbors")
        # ✅ Update Labeled Neighbor Score for Unlabeled Neighbors
        for neighbor in neighbors_dict[node]:
            if y_pred[neighbor] == -1:
                old_entry = next((entry for entry in ordered_set if entry[1] == neighbor), None)

                if old_entry:
                    ordered_set.discard(old_entry)

                num_labeled_by_train = sum(1 for n in neighbors_dict[neighbor] if y_pred[n] != -1 and n in train_labeled_nodes)
                num_labeled_by_propagation = sum(1 for n in neighbors_dict[neighbor] if y_pred[n] != -1 and n not in train_labeled_nodes)
                total_neighbors = len(neighbors_dict[neighbor])

                new_weighted_score = (a4*num_labeled_by_propagation + a5 * num_labeled_by_train) / total_neighbors if total_neighbors > 0 else 0

                ordered_set.add((-new_weighted_score, neighbor))
    print("\n\n\n")

# ✅ Evaluate Accuracy
test_indices = [i for i in range(data.num_nodes) if i not in train_indices]
y_true = data.y[test_indices]
y_pred_test = y_pred[test_indices]

valid_idx = [i for i in range(len(y_pred_test)) if y_pred_test[i] != -1]
if valid_idx:
    final_accuracy = accuracy_score(y_true[valid_idx], y_pred_test[valid_idx])
    print(f"\n🎯 Label Propagation Accuracy: {final_accuracy:.4f}")
else:
    print("\n❌ No valid predictions were made!")



✅ Loaded  dataset with 183 nodes, 557 edges
   - Features: 1703
   - Number of Classes: 5
   - Number of Components: 1
55 1
23 1
9 1
12 1
19 1
3 -17.57540681110165
0 -21.39486875766661
4 -22.501822432851405
1 -20.796830603746862
2 -25.857700270365893

🔹 Labelling Node 1 
   -> Predicted Label: 3 | True Label: 3 ✅
   - Total Neighbors: 3
   - Labeled Neighbors: 3
   - Class 4: 1 neighbors
   - Class 0: 1 neighbors
   - Class 2: 1 neighbors




56 1
23 1
9 1
12 1
19 1
3 -24.788049480779385
0 -19.55581846469786
4 -20.899264296865077
1 -21.49271721568534
2 -24.867778700541674

🔹 Labelling Node 2 
   -> Predicted Label: 0 | True Label: 0 ✅
   - Total Neighbors: 2
   - Labeled Neighbors: 2
   - Class 3: 2 neighbors




56 1
24 1
9 1
12 1
19 1
3 -28.0928018306329
0 -28.727517988623642
4 -30.44883438048324
1 -28.28703636851737
2 -30.274177473735033

🔹 Labelling Node 6 
   -> Predicted Label: 3 | True Label: 4 ❌
   - Total Neighbors: 4
   - Labeled Neighbors: 4
   - Class 3: 1 neighbors
   - C

In [57]:
import torch
import numpy as np
import random
from collections import Counter, defaultdict
from sortedcontainers import SortedSet
from sklearn.metrics import accuracy_score
import networkx as nx
from torch_geometric.utils import to_networkx

def predictclass(data, train_indices, test_indices, a1, a2, a3, a4, a5):
    data.edge_index = torch.cat([data.edge_index, data.edge_index.flip(0)], dim=1)
    train_indices = list(train_indices)
    test_indices = list(test_indices)

    # ✅ Remove duplicate edges
    data.edge_index = torch.unique(data.edge_index, dim=1)

    # ✅ Initialize Prediction Labels
    y_pred = np.full(data.num_nodes, -1, dtype=int)
    for idx in train_indices:
        y_pred[idx] = data.y[idx].item()

    # ✅ Compute Label Distribution
    total_label_counts = Counter(data.y.numpy())
    num_labels = len(set(data.y.numpy()))
    alpha = 1  # Laplace smoothing

    dataset_label_distribution = {
        lbl: (total_label_counts[lbl] + alpha) / (len(data.y) + alpha * num_labels)
        for lbl in total_label_counts.keys()
    }

    train_label_counts = Counter(y_pred[train_indices])

    train_label_distribution = {
        lbl: (train_label_counts.get(lbl, 0) + alpha) / (len(train_indices) + alpha * num_labels)
        for lbl in dataset_label_distribution.keys()
    }

    # ✅ Precompute Neighbors for Each Node
    A = torch.zeros((data.num_nodes, data.num_nodes))
    A[data.edge_index[0], data.edge_index[1]] = 1
    neighbors_dict = {i: set(np.where(A[i] == 1)[0]) for i in range(data.num_nodes)}

    # ✅ Track Instances Assigned to Each Label
    class_instances = defaultdict(set)
    for idx in train_indices:
        class_instances[data.y[idx].item()].add(idx)

    # ✅ Ordered Set for Managing Unlabeled Vertices
    ordered_set = SortedSet()
    train_labeled_nodes = set(train_indices)

    for node in range(data.num_nodes):
        if y_pred[node] == -1:
            labeled_neighbors = [n for n in neighbors_dict[node] if y_pred[n] != -1]
            num_labeled_by_train = sum(1 for n in labeled_neighbors if n in train_labeled_nodes)
            num_labeled_by_propagation = len(labeled_neighbors) - num_labeled_by_train
            total_neighbors = len(neighbors_dict[node])

            if total_neighbors > 0:
                weighted_score = (a4 * num_labeled_by_propagation + a5 * num_labeled_by_train) / total_neighbors
            else:
                weighted_score = 0

            ordered_set.add((-weighted_score, node))

    # ✅ Iteratively Label the Most Constrained Nodes
    while ordered_set:
        _, node = ordered_set.pop(0)

        if y_pred[node] != -1:
            continue  # Skip already labeled nodes

        # Get labeled neighbors
        neighbor_labels = [y_pred[n] for n in neighbors_dict[node] if y_pred[n] != -1]
        label_counts = Counter(neighbor_labels)

        # 🚨 If this node has 0 labeled neighbors
        if not neighbor_labels:
            continue

        # Compute Neighbor Label Distribution
        neighbor_label_counts = Counter(neighbor_labels)
        total_labeled_neighbors = len(neighbor_labels)

        if total_labeled_neighbors > 0:
            neighbor_label_distribution = {
                lbl: neighbor_label_counts[lbl] / total_labeled_neighbors for lbl in neighbor_label_counts.keys()
            }
        else:
            neighbor_label_distribution = {lbl: 0 for lbl in dataset_label_distribution.keys()}

        feature_diffs = {}

        for lbl in dataset_label_distribution.keys():
            if class_instances[lbl]:  # Only compute if we have instances of the label
                instance_features = data.x[list(class_instances[lbl])]
                node_feature = data.x[node].unsqueeze(0)

                if instance_features.numel() > 0:  # ✅ Ensure instance_features is not empty
                    avg_distance = torch.mean(torch.norm(instance_features - node_feature, dim=1)).item()
                else:
                    avg_distance = 0  # ✅ Avoid empty slice issue
                feature_diffs[lbl] = avg_distance
            else:
                feature_diffs[lbl] = 0  # No penalty if no instances exist

        # ✅ Select the Best Label Based on the Adjusted Score
        best_label_candidates = []
        max_value = float('-inf')

        for lbl in dataset_label_distribution.keys():
            score = (
                a1 * train_label_distribution.get(lbl, 0)
                + a2 * neighbor_label_distribution.get(lbl, 0)
                + a3 * feature_diffs[lbl]  # Penalize by average feature distance
            )

            if score - max_value > 0.01:
                best_label_candidates = [lbl]
                max_value = score
            elif abs(score - max_value) <= 0.01:
                best_label_candidates.append(lbl)

        best_label = random.choice(best_label_candidates)
        y_pred[node] = best_label
        class_instances[best_label].add(node)  # Track newly labeled nodes

        # ✅ Update Labeled Neighbor Score for Unlabeled Neighbors
        for neighbor in neighbors_dict[node]:
            if y_pred[neighbor] == -1:
                old_entry = next((entry for entry in ordered_set if entry[1] == neighbor), None)

                if old_entry:
                    ordered_set.discard(old_entry)

                num_labeled_by_train = sum(1 for n in neighbors_dict[neighbor] if y_pred[n] != -1 and n in train_labeled_nodes)
                num_labeled_by_propagation = sum(1 for n in neighbors_dict[neighbor] if y_pred[n] != -1 and n not in train_labeled_nodes)
                total_neighbors = len(neighbors_dict[neighbor])

                new_weighted_score = (a4 * num_labeled_by_propagation + a5 * num_labeled_by_train) / total_neighbors if total_neighbors > 0 else 0

                ordered_set.add((-new_weighted_score, neighbor))

    # ✅ Convert to PyTorch Tensor for Neural Network
    y_pred_test = torch.tensor(y_pred[test_indices], dtype=torch.float)

    # ✅ Compute Initial Classification Accuracy
    y_true = data.y[test_indices]
    valid_idx = [i for i in range(len(y_pred_test)) if y_pred_test[i] != -1]
    if valid_idx:
        initial_accuracy = accuracy_score(y_true[valid_idx], y_pred_test[valid_idx])
    else:
        initial_accuracy = 0.0  # No valid predictions

    return y_pred_test, initial_accuracy



In [58]:
#dataset = WebKB(root='data', name='Cornell')
dataset = WebKB(root='data', name='texas')
#dataset = WikipediaNetwork(root='data', name='chameleon')
#dataset = WikipediaNetwork(root='data', name='squirrel')
data = dataset[0]



# Count occurrences of each class label
class_counts = torch.bincount(data.y)

# Print the number of elements in each class
for class_id, count in enumerate(class_counts):
    print(f"Class {class_id}: {count} elements")
a1=0.5
a2=-6
a3=-2
a4=1
a5=3
indices = np.arange(data.num_nodes)  # NumPy array of indices [0, 1, 2, ..., num_nodes-1]

train_idx, test_idx = train_test_split(indices, test_size=0.3, random_state=42)

# Further split train_idx into train and validation sets (e.g., 20% of train goes to validation)
train_idx, val_idx = train_test_split(train_idx, test_size=0.2, random_state=42)
print(predictclass(data,train_idx,valid_idx,a1,a2,a3,a4,a5))

Class 0: 33 elements
Class 1: 1 elements
Class 2: 18 elements
Class 3: 101 elements
Class 4: 30 elements
(tensor([3., 0., 1., 3., 4., 3., 3., 0., 3., 3., 1., 3., 3., 3., 3., 1., 3., 3.,
        1., 3., 0., 3., 4., 3., 3., 1., 3., 3., 0., 1., 3., 3., 4., 3., 4., 3.,
        2., 3., 3., 3., 0., 4., 1., 3., 3., 3., 3., 2., 0., 3., 0., 1., 3., 3.,
        3., 4., 1., 2., 2., 4., 1., 4., 3., 3., 3.]), 0.7692307692307693)


Processing...
Done!


In [73]:
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
import warnings
warnings.simplefilter("error", RuntimeWarning)  # Convert warnings to errors
#dataset = WebKB(root='data', name='Cornell')
dataset = WebKB(root='data', name='texas')
#dataset = WikipediaNetwork(root='data', name='chameleon')
#dataset = WikipediaNetwork(root='data', name='squirrel')
#dataset = WebKB(root='data', name='Wisconsin')


data = dataset[0]

num_nodes = data.x.shape[0]
indices = np.arange(num_nodes)

# Split into training (70%) and testing (30%) randomly
train_idx, test_idx = train_test_split(indices, test_size=0.4, random_state=42)

# Further split train_idx into train and validation sets (e.g., 20% of train goes to validation)
train_idx, val_idx = train_test_split(train_idx, test_size=0.5, random_state=42)

# Function to optimize (negate accuracy since we minimize)
# Function to optimize (maximize train accuracy by minimizing negative accuracy)

def random_search(model,data, train_idx,val_idx, num_samples=100):
    best_params = None
    best_score = float('-inf')

    for _ in range(num_samples):
        a1 = random.uniform(0, 10)     # a1 must be positive
        a2 = random.uniform(-10, -0.01)  # a2 must be negative
        a3 = random.uniform(-10, -0.01)  # a3 must be negative
        a5 = random.uniform(0, 10)     # a5 must be positive
        a4 = random.uniform(0,a5 )    # a4 must be positive and a4 >= a5

        params = [a1, a2, a3, a4, a5]


        _,score = model(data,train_idx,val_idx,params[0],params[1],params[2],params[3],params[4])
        if score > best_score:
            best_score = score
            best_params = params

    return best_params, best_score



# Initial guesses for a1, a2, a3, a4, a5
#initial_params = [0.5,-5,-2,1,3]

# Bounds for parameters
 # Adjust bounds as needed

# Optimize using scipy
result = random_search(predictclass,data,train_idx,val_idx,num_samples=10)[0]
#print(result)
# Get best parameters
best_a1, best_a2, best_a3, best_a4, best_a5 = result[0],result[1],result[2],result[3],result[4]

# Evaluate on test set
predicted_classes, final_accuracy = predictclass(data,train_idx,test_idx, best_a1, best_a2, best_a3, best_a4, best_a5)

# Print results
print("Optimal Parameters:", result)
print("Final Accuracy on Test Data:", final_accuracy)

Optimal Parameters: [3.6132187229948465, -3.128579846326563, -4.583218924436441, 5.455283683331452, 5.849375271509487]
Final Accuracy on Test Data: 0.581081081081081


In [78]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn as pyg_nn
from torch_geometric.utils import to_dense_adj
from sklearn.metrics import accuracy_score

class GraphAugmenter(nn.Module):
    def __init__(self, in_dim, hidden_dim):
        super(GraphAugmenter, self).__init__()
        self.gnn = pyg_nn.GCNConv(in_dim, hidden_dim)
        self.edge_mlp = nn.Sequential(
            nn.Linear(2 * hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )

    def forward(self, x, edge_index):
        node_emb = self.gnn(x, edge_index)
        row, col = edge_index
        edge_feat = torch.cat([node_emb[row], node_emb[col]], dim=-1)
        edge_weights = self.edge_mlp(edge_feat).squeeze()
        print("Edge Weights Shape:", edge_weights.shape)
        return edge_weights

class HeterophilicNodeClassifier(nn.Module):
    def __init__(self, in_dim, hidden_dim, num_classes):
        super(HeterophilicNodeClassifier, self).__init__()

        self.augmenter = GraphAugmenter(in_dim, hidden_dim)
        self.gnn_high = pyg_nn.GCNConv(in_dim, hidden_dim)
        self.offset_mlp = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        self.final_mlp = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x, edge_index, train_indices, test_indices, initial_test_preds=None):
        print("Forward pass started")
        print("Input shape:", x.shape)
        print("Edge index shape:", edge_index.shape)

        edge_weights = self.augmenter(x, edge_index)
        print("Edge weights computed")

        # High-pass GNN for heterophilic graphs
        h = self.gnn_high(x, edge_index, edge_weights)
        print("GNN high output shape:", h.shape)

        # Offset term to escape local minima
        offset = self.offset_mlp(x)
        print("Offset shape:", offset.shape)

        # Adaptive aggregation
        logits = self.final_mlp(h + offset)
        print("Logits shape:", logits.shape)

        # Expand initial_test_preds to match full node size
        if initial_test_preds is not None:
          print("Initial predictions received, expanding...")
          expanded_preds = torch.zeros_like(logits)

          train_indices = torch.tensor(train_indices, dtype=torch.long, device=logits.device) if isinstance(train_indices, list) else train_indices
          test_indices = torch.tensor(test_indices, dtype=torch.long, device=logits.device) if isinstance(test_indices, list) else test_indices

          # Ensure `initial_test_preds` is a proper tensor
          if not isinstance(initial_test_preds, torch.Tensor):
              initial_test_preds = torch.tensor(initial_test_preds, dtype=torch.long, device=logits.device).clone().detach()

          print("Train Indices Shape:", train_indices.shape)
          print("Test Indices Shape:", test_indices.shape)
          print("Initial Test Predictions Shape:", initial_test_preds.shape)

          # Assign ground truth labels for training nodes
          expanded_preds[train_indices] = F.one_hot(data.y[train_indices].long(), num_classes=logits.shape[1]).float()

          # Ensure initial_test_preds aligns with test_indices correctly
          if initial_test_preds.shape[0] == len(test_indices):
              expanded_preds[test_indices] = F.one_hot(initial_test_preds.long(), num_classes=logits.shape[1]).float()
          else:
              print(f"Warning: initial_test_preds shape {initial_test_preds.shape} does not match test_indices length {len(test_indices)}")
              print("Attempting to reshape initial predictions...")

              full_initial_preds = torch.full((logits.shape[0],), -1, dtype=torch.long, device=logits.device)
              full_initial_preds[test_indices] = initial_test_preds

              valid_mask = full_initial_preds != -1
              expanded_preds[valid_mask] = F.one_hot(full_initial_preds[valid_mask].long(), num_classes=logits.shape[1]).float()

          print("Expanded predictions shape:", expanded_preds.shape)
          logits = logits + 0.1 * (logits - expanded_preds)


        print("Forward pass completed")
        return F.log_softmax(logits, dim=1)

# Training and Evaluation Code
def train_and_evaluate(model, data, train_indices, test_indices, initial_preds, epochs=100, lr=0.01):
    criterion = nn.NLLLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        print(f"Epoch {epoch+1} training started")
        logits = model(data.x, data.edge_index, train_indices, test_indices, initial_preds)
        print("Logits computed")

        loss = criterion(logits[train_indices], data.y[train_indices])
        print(f"Loss computed: {loss.item()}")

        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

    # Evaluation
    model.eval()
    with torch.no_grad():
        print("Evaluation started")
        logits = model(data.x, data.edge_index, train_indices, test_indices, initial_preds)
        predicted_classes = torch.argmax(logits, dim=1)
        test_predictions = predicted_classes[test_indices]

        final_accuracy = accuracy_score(data.y[test_indices].cpu().numpy(), test_predictions.cpu().numpy())

    print(f"Final Test Accuracy: {final_accuracy:.4f}")
    print("\nTest Node Predictions:")
    for i, node in enumerate(test_indices):
        print(f"Node {node}: Predicted Class = {test_predictions[i].item()}, True Class = {data.y[node].item()}")

    return final_accuracy, test_predictions


In [79]:
# Generate initial predictions and accuracy
initial_preds, initial_accuracy = predictclass(data, train_idx, test_idx, best_a1, best_a2, best_a3, best_a4, best_a5)

print(f"Initial Classification Accuracy: {initial_accuracy:.4f}")

# Create the model
model = HeterophilicNodeClassifier(in_dim=data.x.shape[1], hidden_dim=32, num_classes=len(set(data.y.tolist())))

# Train the model before inference
final_accuracy, test_predictions = train_and_evaluate(model, data, train_idx, test_idx, initial_preds, epochs=100, lr=0.01)

# Now you can use the trained model for inference
model.eval()
with torch.no_grad():
    logits = model(data.x, data.edge_index, train_indices, test_indices, initial_test_preds=initial_preds)


    predicted_classes = torch.argmax(logits, dim=1)


print(f"Final Classification Accuracy: {final_accuracy:.4f}")


Initial Classification Accuracy: 0.5811
Epoch 1 training started
Forward pass started
Input shape: torch.Size([183, 1703])
Edge index shape: torch.Size([2, 574])
Edge Weights Shape: torch.Size([574])
Edge weights computed
GNN high output shape: torch.Size([183, 32])
Offset shape: torch.Size([183, 32])
Logits shape: torch.Size([183, 5])
Initial predictions received, expanding...
Train Indices Shape: (54,)
Test Indices Shape: (74,)
Initial Test Predictions Shape: torch.Size([74])
Expanded predictions shape: torch.Size([183, 5])
Forward pass completed
Logits computed
Loss computed: 1.7474135160446167
Epoch 1, Loss: 1.7474
Epoch 2 training started
Forward pass started
Input shape: torch.Size([183, 1703])
Edge index shape: torch.Size([2, 574])
Edge Weights Shape: torch.Size([574])
Edge weights computed
GNN high output shape: torch.Size([183, 32])
Offset shape: torch.Size([183, 32])
Logits shape: torch.Size([183, 5])
Initial predictions received, expanding...
Train Indices Shape: (54,)
Test 

RuntimeError: shape mismatch: value tensor of shape [74] cannot be broadcast to indexing result of shape [65]